In [1]:
from blob_env import BlobEnv
import numpy as np
from util import *

from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment

ImportError: cannot import name 'BlobEnv'

In [1]:
#train_env = BlobEnv()
#obs = train_env.reset()[0]
#print(str(train_env.player))
#print(str(train_env.enemy))
#print((train_env.player-train_env.food) + (train_env.player-train_env.enemy))
#(1,0) + (1,0)

In [3]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

In [4]:
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"} 
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

In [5]:
tf_env = tf_py_environment.TFPyEnvironment(suite_gym.wrap_env(BlobEnv()))
eval_tf_env = tf_py_environment.TFPyEnvironment(suite_gym.wrap_env(BlobEnv()))

In [6]:
tf_env.time_step_spec()

TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=BoundedTensorSpec(shape=(1, 10, 10, 3), dtype=tf.uint8, name='observation', minimum=array(0, dtype=uint8), maximum=array(255, dtype=uint8)))

In [7]:
tf_env.observation_spec()

BoundedTensorSpec(shape=(1, 10, 10, 3), dtype=tf.uint8, name='observation', minimum=array(0, dtype=uint8), maximum=array(255, dtype=uint8))

In [8]:
tf_env.action_spec()

BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0), maximum=array(8))

In [10]:
fc_layer_params = (100,)

q_net = q_network.QNetwork(
    tf_env.observation_spec(),
    tf_env.action_spec(),
    fc_layer_params=fc_layer_params)

In [11]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

In [12]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

In [13]:
random_policy = random_tf_policy.RandomTFPolicy(tf_env.time_step_spec(),
                                                tf_env.action_spec())

In [14]:
example_environment = tf_py_environment.TFPyEnvironment(suite_gym.wrap_env(BlobEnv()))

In [15]:
time_step = example_environment.reset()

In [16]:
random_policy.action(time_step)

ValueError: Received a mix of batched and unbatched Tensors, or Tensors are not compatible with Specs.  num_outer_dims: 1.
Saw tensor_shapes:
   TimeStep(step_type=TensorShape([1]), reward=TensorShape([1]), discount=TensorShape([1]), observation=TensorShape([1, 10, 10, 3]))
And spec_shapes:
   TimeStep(step_type=TensorShape([]), reward=TensorShape([]), discount=TensorShape([]), observation=TensorShape([1, 10, 10, 3]))

In [23]:
example_environment = tf_py_environment.TFPyEnvironment(suite_gym.load('CartPole-v1'))
example_environment.time_step_spec().step_type
random_policy = random_tf_policy.RandomTFPolicy(example_environment.time_step_spec(),
                                                example_environment.action_spec())

In [24]:
random_policy.action(time_step)

TypeError: Tensor dtypes do not match spec dtypes:
TimeStep(step_type=tf.int32, reward=tf.float32, discount=tf.float32, observation=tf.uint8)
vs.
TimeStep(step_type=tf.int32, reward=tf.float32, discount=tf.float32, observation=tf.float32)

In [ ]:

def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics